# Relevant Bits of Code for Napari 

## Data Preparation
### Unstacking 

This code snippet helps unstack a stacked tif file with txy format. 

In [ ]:
import imageio
import os

def extract_tiff_frames_with_imageio(tiff_path, output_folder):
    reader = imageio.get_reader(tiff_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for i, im in enumerate(reader):
        if im.ndim == 2:  
            imageio.imwrite(f"{output_folder}/frame_{str(i+1).zfill(4)}.tif", im)
        elif im.ndim == 3:  
            for j, slice in enumerate(im):
                imageio.imwrite(f"{output_folder}/frame_{str(i+1).zfill(4)}_slice_{str(j+1).zfill(4)}.tif", slice)
        else:
            print(f"Frame {i+1} has unsupported dimensions: {im.ndim}")

data = r"D:\documents\digibio\data\image_intensities_(well)_unstacked"
file = r"D:\documents\digibio\data\amdtrk\x1_imageintensity.tif"

extract_tiff_frames_with_imageio(file, data)

## Napari Launch and Usage 

This code snippet allows visualization of unstacked images and masks, as well as the respective tracking data corresponding to each X and Y position of the tracked cells.

In [ ]:
import pandas as pd
import numpy as np
import os
from skimage import io
import napari

%gui qt

image_dir = # images unstacked folder
mask_dir_1 = # optical unstacked folder
mask_dir_2 = # cellpose unstacked folder
data_path = # tracking data

image_files = sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
frames = [io.imread(os.path.join(image_dir, image_file)) for image_file in image_files]
stacked_frames = np.stack(frames)
viewer = napari.view_image(stacked_frames)
mask_dir_1_files = sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
frames_mask_dir_1 = [io.imread(os.path.join(mask_dir_1, mask_dir_1_files)) for mask_dir_1_files in mask_dir_1_files]
stacked_frames_mask_dir_1 = np.stack(frames_mask_dir_1)
viewer.add_image(stacked_frames_mask_dir_1)
mask_dir_2_files = sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
frames_mask_dir_2 = [io.imread(os.path.join(mask_dir_2, mask_dir_2_files)) for mask_dir_2_files in mask_dir_2_files]
stacked_frames_mask_dir_2 = np.stack(frames_mask_dir_2)
viewer.add_labels(stacked_frames_mask_dir_2)

dataf = pd.read_csv(data_path, low_memory=False)
datafilter = dataf.copy()
datafilter = datafilter.dropna(subset=['id'])
scale_factor = 1
datafilter["SCALED_Y"] = datafilter["posyA"] * scale_factor
datafilter["SCALED_X"] = datafilter["posxB"] * scale_factor
track3 = datafilter.reset_index()
viewer.add_tracks(track3[["id", "frame", "SCALED_X", "SCALED_Y"]])
viewer.add_tracks(track3[["id", "frame", "SCALED_X", "SCALED_Y"]])